In [1]:
from PIL import Image
from os import listdir
from os.path import isdir
from numpy import asarray, expand_dims
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

In [2]:
# Carregando base de dados 
!git clone https://github.com/snitraMnolraM/Criando_Sistema_de_Reconhecimento_Facial_do_Zero.git

Cloning into 'Criando_Sistema_de_Reconhecimento_Facial_do_Zero'...
remote: Enumerating objects: 5806, done.
remote: Counting objects: 100% (1004/1004), done.
remote: Compressing objects: 100% (996/996), done.
remote: Total 5806 (delta 11), reused 978 (delta 8), pack-reused 4802
Receiving objects: 100% (5806/5806), 435.92 MiB | 14.96 MiB/s, done.
Resolving deltas: 100% (59/59), done.
Checking out files: 100% (5835/5835), done.


In [3]:
def load_face(filename):
  image = Image.open(filename) # caminho do arquivo
    
  image = image.convert('RGB')   

  return asarray(image)

In [4]:
#carregando as faces do diretorio 
def load_faces(directory_src):
  faces = list()

  # interando os arquivos
  for filename in listdir(directory_src):

    path = directory_src + filename

    try:
      faces.append(load_face(path))
    except:
      print("Erro na imagem {}".format(path))

  return faces  


In [5]:
## carregando todos os dataset de imagens de faces

def load_fotos(directory_src):
  
  X, y = list(), list()

  # interar pastas por classes
  for subdir in listdir(directory_src):

    #path
    path = directory_src + subdir + "/"
    
    if not isdir(path):
      continue

    faces = load_faces(path)
    
    labels = [subdir for _ in range(len(faces))]

    #sumarizar progresso
    print('>Carregagadas %d faces da classe: %s' % (len(faces), subdir))


    X.extend(faces)
    y.extend(labels)

  return asarray(X), asarray(y)

## **Carregando todas as imagens**

In [20]:
trainX, trainy = load_fotos(directory_src = '/content/Criando_Sistema_de_Reconhecimento_Facial_do_Zero/dataset/Faces/')

>Carregagadas 192 faces da classe: Santos
>Carregagadas 624 faces da classe: Gabigol
>Carregagadas 216 faces da classe: Joao_Gomes
>Carregagadas 228 faces da classe: Rodinei
>Carregagadas 144 faces da classe: Thiago_Maia
>Carregagadas 156 faces da classe: Filipe_Luis
>Carregagadas 178 faces da classe: Everton_Ribeiro
>Carregagadas 220 faces da classe: David_Luiz
>Carregagadas 160 faces da classe: Pedro
>Carregagadas 228 faces da classe: Leo_Pereira
>Carregagadas 264 faces da classe: Arrascaeta
>Carregagadas 691 faces da classe: Desconhecido


In [21]:
trainX.shape

(3301, 160, 160, 3)

In [22]:
model = load_model('/content/Criando_Sistema_de_Reconhecimento_Facial_do_Zero/facenet_keras.h5', compile=False)

In [ ]:
#model.summary()

## **Função Geradora de Embeddings**

In [23]:
def get_embedding(model, face_pixels):

  #Padronização
  mean, std = face_pixels.mean(), face_pixels.std()
  face_pixels = (face_pixels - mean)/std

  #Transformando a face em 1 único exemplo
  
  ## (160, 160) -> (1, 160, 160)
  
  samples = expand_dims(face_pixels, axis=0)
  
  # realizar a predição gerando o embeddig
  yhat = model.predict(samples)

  return yhat[0]



In [24]:
newTrainX = list()

In [25]:
for face_pixels in trainX:
  embedding = get_embedding(model, face_pixels)
  newTrainX.append(embedding)


In [26]:
newTrainX = asarray(newTrainX)

In [27]:
newTrainX.shape

(3301, 128)

In [28]:
df = pd.DataFrame(data=newTrainX)

In [29]:
df

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-0.828319,0.347261,-0.102824,1.110965,-1.069658,-1.603276,-1.507681,0.915434,-0.243845,0.637408,...,0.662168,1.479728,-1.488623,-0.717623,-2.380243,-1.364763,-1.148158,2.060484,0.687899,-0.256705
1,-1.311465,0.473761,1.184975,0.837722,-0.986167,0.088024,0.002401,-0.292260,-0.035554,-0.100241,...,-0.903820,0.910073,-0.683683,0.295436,-0.975720,-0.688874,-1.835845,0.510611,0.478889,0.751306
2,-0.904878,0.910872,1.481133,0.692403,-0.410299,-0.629313,0.169015,-0.826408,0.338080,-0.622369,...,-0.398740,1.091286,-0.867828,0.484684,-1.222550,-0.826724,-1.195427,0.961842,0.111536,1.033851
3,-0.864151,-0.186849,-0.368660,0.156282,-0.743199,-0.683198,-0.935689,0.415689,-0.730534,0.412707,...,0.327890,1.315781,-1.770793,2.016524,-2.041472,0.054812,-0.561298,-0.897075,1.275198,0.976329
4,0.042400,1.201213,0.692493,0.836768,-0.928427,-0.563588,0.669547,-0.702055,0.199747,0.524555,...,-0.853744,1.801917,-1.372162,0.350625,-1.616485,-0.449506,-2.045272,1.223271,-0.037299,-0.033328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3296,2.214622,-0.892696,-0.025543,0.264256,1.027872,-0.080506,0.330456,0.101382,0.499247,0.319037,...,-1.090409,1.105542,2.018212,-0.337148,0.122312,-0.843702,-1.610231,0.203335,-0.350950,-1.324592
3297,0.603437,-0.330001,-1.477440,-0.268367,-0.484546,-0.088676,-0.438667,-0.457203,0.870595,-0.171819,...,0.384457,-1.049451,1.278901,-0.641879,-0.128639,0.984689,-0.782011,-0.681047,-0.074032,-0.425313
3298,-0.220303,0.037024,0.259256,0.941429,1.187605,-0.069720,0.156543,-0.039570,-0.056281,-0.235721,...,0.390918,-1.640661,-0.386480,-0.517925,0.973711,-0.087621,-0.769330,-1.861863,0.122753,0.353206
3299,1.087479,0.755220,-1.560251,0.233360,-1.258553,2.079727,0.106518,0.019853,0.534495,0.649822,...,-0.869255,-0.153888,-0.204704,0.093621,1.049766,0.505572,0.370888,-0.188503,0.079960,0.963694


In [30]:
df['target'] = trainy

In [31]:
df

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,target
0,-0.828319,0.347261,-0.102824,1.110965,-1.069658,-1.603276,-1.507681,0.915434,-0.243845,0.637408,...,1.479728,-1.488623,-0.717623,-2.380243,-1.364763,-1.148158,2.060484,0.687899,-0.256705,Santos
1,-1.311465,0.473761,1.184975,0.837722,-0.986167,0.088024,0.002401,-0.292260,-0.035554,-0.100241,...,0.910073,-0.683683,0.295436,-0.975720,-0.688874,-1.835845,0.510611,0.478889,0.751306,Santos
2,-0.904878,0.910872,1.481133,0.692403,-0.410299,-0.629313,0.169015,-0.826408,0.338080,-0.622369,...,1.091286,-0.867828,0.484684,-1.222550,-0.826724,-1.195427,0.961842,0.111536,1.033851,Santos
3,-0.864151,-0.186849,-0.368660,0.156282,-0.743199,-0.683198,-0.935689,0.415689,-0.730534,0.412707,...,1.315781,-1.770793,2.016524,-2.041472,0.054812,-0.561298,-0.897075,1.275198,0.976329,Santos
4,0.042400,1.201213,0.692493,0.836768,-0.928427,-0.563588,0.669547,-0.702055,0.199747,0.524555,...,1.801917,-1.372162,0.350625,-1.616485,-0.449506,-2.045272,1.223271,-0.037299,-0.033328,Santos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3296,2.214622,-0.892696,-0.025543,0.264256,1.027872,-0.080506,0.330456,0.101382,0.499247,0.319037,...,1.105542,2.018212,-0.337148,0.122312,-0.843702,-1.610231,0.203335,-0.350950,-1.324592,Desconhecido
3297,0.603437,-0.330001,-1.477440,-0.268367,-0.484546,-0.088676,-0.438667,-0.457203,0.870595,-0.171819,...,-1.049451,1.278901,-0.641879,-0.128639,0.984689,-0.782011,-0.681047,-0.074032,-0.425313,Desconhecido
3298,-0.220303,0.037024,0.259256,0.941429,1.187605,-0.069720,0.156543,-0.039570,-0.056281,-0.235721,...,-1.640661,-0.386480,-0.517925,0.973711,-0.087621,-0.769330,-1.861863,0.122753,0.353206,Desconhecido
3299,1.087479,0.755220,-1.560251,0.233360,-1.258553,2.079727,0.106518,0.019853,0.534495,0.649822,...,-0.153888,-0.204704,0.093621,1.049766,0.505572,0.370888,-0.188503,0.079960,0.963694,Desconhecido


In [32]:
df.to_csv('faces1.csv')